# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

df = pd.read_csv(url)

df.head()
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [2]:
filtered_df = df[(df['Total Claim Amount'] > 1000) & (df['Response'] == 'Yes')]

print (filtered_df)

       Unnamed: 0 Customer       State  Customer Lifetime Value Response  \
189           189  OK31456  California             11009.130490      Yes   
236           236  YJ16163      Oregon             11009.130490      Yes   
419           419  GW43195      Oregon             25807.063000      Yes   
442           442  IP94270     Arizona             13736.132500      Yes   
587           587  FJ28407  California              5619.689084      Yes   
...           ...      ...         ...                      ...      ...   
10351       10351  FN44127      Oregon              3508.569533      Yes   
10373       10373  XZ64172      Oregon             10963.957230      Yes   
10487       10487  IX60941      Oregon              3508.569533      Yes   
10565       10565  QO62792      Oregon              7840.165778      Yes   
10708       10708  CK39096      Oregon              5619.689084      Yes   

       Coverage             Education Effective To Date EmploymentStatus  \
189     Pre

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [3]:
# Filtering customers who responded "Yes" to the last marketing campaign
responded_yes_df = df[df['Response'] == 'Yes']

# Grouping by policy_type and gender, and calculating the average total_claim_amount
average_claim_by_policy_gender = responded_yes_df.groupby(['Policy Type', 'Gender'])['Total Claim Amount'].mean()

print(average_claim_by_policy_gender)


Policy Type     Gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: Total Claim Amount, dtype: float64


### Conclusions:

From the output of the code, we can see that:

-Differences in Claim Amounts by Policy Type:
    Certain policies are associated with customers who claim more when they do respond positively to marketing, such as personal auto.

-Impact of Gender:
    We can see that for female users the total claim amount is higher for corporate and special auto.

-Marketing Campaign Effectiveness:
    Understanding how these variables interact with the claim amounts could also give insights into how different customer segments (based on policy type and gender) respond in value to marketing efforts, which could inform targeted campaigns in the future.


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [4]:
customers_by_state = df['State'].value_counts()  # Count the number of customers by state

states_with_more_than_500_customers = customers_by_state[customers_by_state > 500]  # Filter for states with more than 500 customers

print(states_with_more_than_500_customers)


State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [5]:
# Group by education and gender, and calculate max, min, and median for customer_lifetime_value
# clv = customer_lifetime_value
clv_stats_by_education_gender = df.groupby(['Education', 'Gender'])['Customer Lifetime Value'].agg(['max', 'min', 'median'])

# agg(['max', 'min', 'median']) calculates the maximum, minimum, and median customer_lifetime_value (CLV) for each group.


print(clv_stats_by_education_gender)

                                     max          min       median
Education            Gender                                       
Bachelor             F       73225.95652  1904.000852  5640.505303
                     M       67907.27050  1898.007675  5548.031892
College              F       61850.18803  1898.683686  5623.611187
                     M       61134.68307  1918.119700  6005.847375
Doctor               F       44856.11397  2395.570000  5332.462694
                     M       32677.34284  2267.604038  5577.669457
High School or Below F       55277.44589  2144.921535  6039.553187
                     M       83325.38119  1940.981221  6286.731006
Master               F       51016.06704  2417.777032  5729.855012
                     M       50568.25912  2272.307310  5579.099207


### Conclusions

-Maximum Customer Lifetime Value:
    The maximum CLV within each education level and gender may indicate the highest potential value a customer from that segment can bring. Higher values could indicate customers with greater engagement or loyalty.

-Minimum Customer Lifetime Value:
    The minimum CLV values can show the least engaged or low-value customers. This could help identify customer segments where there might be an opportunity to increase their lifetime value through targeted campaigns.

-Median Customer Lifetime Value:
    The median CLV provides a better representation of the central tendency compared to the average, especially if there are outliers. It can help understand the typical customer value for each education level and gender combination.

These insights could be valuable for tailoring marketing strategies. For example, if certain education levels and genders tend to have higher CLV, efforts can be focused on attracting or retaining such customers. Additionally, understanding the spread between maximum and minimum values can highlight the variability in customer value within each group.


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [35]:
# To analyze the number of policies sold by state and month and present the data in a table where months are columns and states are rows, 
# we use a pivot table. It is necessary to extract the month from the date column.

In [8]:
print(df['Effective To Date'])

0       2011-02-18
1       2011-01-18
2       2011-02-10
3       2011-01-11
4       2011-01-17
           ...    
10905   2011-01-19
10906   2011-01-06
10907   2011-02-06
10908   2011-02-13
10909   2011-01-08
Name: Effective To Date, Length: 10910, dtype: datetime64[ns]


In [9]:
# Ensure the date column is in datetime format 

df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])  

# Extract the month from the date column
df['month'] = df['Effective To Date'].dt.month

# Create a pivot table with states as rows and months as columns, counting the number of polic

policies_by_state_month = df.pivot_table(
    index='State', 
    columns='month', 
    values=['Policy'],  
    aggfunc='count', 
    fill_value=0
)

# Display the pivot table
print(policies_by_state_month)

           Policy      
month           1     2
State                  
Arizona      1008   929
California   1918  1634
Nevada        551   442
Oregon       1565  1344
Washington    463   425


In [10]:
print(df['Effective To Date'])

0       2011-02-18
1       2011-01-18
2       2011-02-10
3       2011-01-11
4       2011-01-17
           ...    
10905   2011-01-19
10906   2011-01-06
10907   2011-02-06
10908   2011-02-13
10909   2011-01-08
Name: Effective To Date, Length: 10910, dtype: datetime64[ns]


Explanation

    Convert Date Column: The pd.to_datetime() function converts the date column to a datetime object, which is necessary for extracting the month.
    Extract the Month: df['month'] = df['date'].dt.month extracts the month as a separate column.
    Create Pivot Table: The pivot_table() function organizes the data with state as the rows and month as the columns, counting the number of occurrences of Policy.
    Fill Missing Values: Using fill_value=0 ensures that any missing combinations are displayed as zeros.

Result

The output is a table where each row corresponds to a state, each column corresponds to a month (1-12), and the cells contain the count of policies sold for that state in each month. This format allows the marketing team to easily analyze seasonal trends or state-specific variations in policy sales.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [11]:
#To create a DataFrame showing the number of policies sold by month for the top 3 states with the highest total number of policies sold, follow the steps below:

#    Group the data by state and month, and count the number of policies sold.
#    Identify the top 3 states with the highest total number of policies sold.
#    Filter the DataFrame to include only those top 3 states.
#    Create a pivot table to display the number of policies sold by month for these top 3 states.



# Group by state and month, and count the number of policies sold

policies_by_state_month = df.groupby(['State', 'month']).size().reset_index(name='policy_count')
print(policies_by_state_month)

# Find the top 3 states with the highest total number of policies sold
top_states = policies_by_state_month.groupby('State')['policy_count'].sum().nlargest(3).index
print(top_states)

# Filter the data to include only the top 3 states
top_states_data = policies_by_state_month[policies_by_state_month['State'].isin(top_states)]
print(top_states_data)



        State  month  policy_count
0     Arizona      1          1008
1     Arizona      2           929
2  California      1          1918
3  California      2          1634
4      Nevada      1           551
5      Nevada      2           442
6      Oregon      1          1565
7      Oregon      2          1344
8  Washington      1           463
9  Washington      2           425
Index(['California', 'Oregon', 'Arizona'], dtype='object', name='State')
        State  month  policy_count
0     Arizona      1          1008
1     Arizona      2           929
2  California      1          1918
3  California      2          1634
6      Oregon      1          1565
7      Oregon      2          1344


In [12]:
# Create a pivot table to display the data with months as columns and states as rows
pivot_table_top_states = top_states_data.pivot_table(
    index='State', 
    columns='month', 
    values=['policy_count'], 
    aggfunc='sum', 
    fill_value=0
)

# Display the pivot table
print(pivot_table_top_states)

           policy_count      
month                 1     2
State                        
Arizona            1008   929
California         1918  1634
Oregon             1565  1344


Explanation

    Group and Count Policies: The groupby(['state', 'month']).size() counts the number of policies sold for each state and month combination.
    Find Top 3 States: nlargest(3) identifies the top 3 states based on the total count of policies.
    Filter for Top States: isin() selects rows corresponding to the top 3 states.
    Create Pivot Table: The pivot_table() function displays the number of policies sold by month for each of the top 3 states.

Result

The output will be a table where each row represents one of the top 3 states, each column represents a month (1-12), and the cells contain the number of policies sold in that month for the respective state. This table helps visualize the monthly distribution of policy sales for the most active states.


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [39]:
#To analyze the effect of different marketing channels on the customer response rate, we can unpivot the data using melt() if the marketing channels are represented in separate columns. After unpivoting, we can calculate the response rate for each channel. Here’s how to do this:

#    Melt the Data: Use melt() to unpivot the columns representing different marketing channels into a long-format DataFrame.
#    Filter the Data: Focus on customers who responded "Yes" to calculate the response rate for each channel.
#    Group and Calculate the Response Rate: Calculate the proportion of positive responses ("Yes") for each marketing channel.

In [14]:


melted_df = pd.melt(df, id_vars=['Response'], value_vars=['Sales Channel'], 
                    var_name='marketing_channel', value_name='channel_response')

# Filter rows where response = yes
response_yes = melted_df[melted_df['Response'] == 'Yes']

# Calculate the response rate for each channel
response_rate_by_channel = response_yes.groupby('marketing_channel')['channel_response'].count() / melted_df.groupby('marketing_channel')['channel_response'].count()

# Convert to percentage
response_rate_by_channel = response_rate_by_channel * 100

# Show the table with the rate of responses
print(response_rate_by_channel)

marketing_channel
Sales Channel    13.437214
Name: channel_response, dtype: float64


Explanation

    Melt the Data:
        pd.melt() is used to unpivot columns that represent different marketing channels (e.g., channel_1, channel_2, etc.). This long-format DataFrame has a new marketing_channel column containing the channel names, and channel_response containing the corresponding values.
    Filter Responses:
        Filter the unpivoted DataFrame for rows where response is "Yes" to focus on customers who positively responded to the marketing.
    Calculate Response Rate:
        Calculate the response rate by dividing the count of "Yes" responses by the total number of responses (both "Yes" and "No") for each marketing channel.
    Convert to Percentage:
        Multiply by 100 to express the response rates as percentages for easier interpretation.

Result

The output will be a table showing the response rates for each marketing channel. The marketing team can use this to determine which channels are more effective at generating positive customer responses. Higher response rates for a particular channel might indicate a more successful approach, suggesting where to focus future marketing efforts.


In [ ]:
# your code goes here